### Imports


In [42]:
import numpy as np
from scipy.stats import chi2
from random import random, gauss, gammavariate
from tabulate import tabulate
from math import exp, comb, log, sqrt, erf


### Funciones generales


In [43]:
def generar_muestra(generador, n):
    """
    Genera una muestra de tamaño n de una variable aleatoria
    Args:
        generador:    Generador de la variable aleatoria
        n        :    Tamaño de la muestra
    """
    datos = []
    for _ in range(n):
        datos.append(generador())
    return datos


def generar_binomial(N, P):
    """
    Genera un valor de la variable aleatoria binomial usando el metodo de la 
    transformada inversa
    Args:
        N: Cantidad de ensayos independientes
        P: Probabilidad de exito de ensayo
    """
    prob = (1 - P) ** N
    c = P / (1 - P)
    F = prob
    i = 0
    U = random()
    while U >= F:
        prob *= c * (N-i) / (i+1)
        F += prob
        i += 1
    return i


def generar_exp(Lambda):
    """
    Genera una variable aleatoria exponencial usando el metodo de la transformada
    inversa
    Args:
        Lambda: Parametro de la variable aleatoria exponencial
    """
    U = random()
    return -log(U) / Lambda


def uniforme_d(n, m):
    """
    Genera numeros aleatorios entre n y m (inclusive) 
    """
    U = int(random() * (m - n + 1))  # 0 <= U < (m-n+1) Equivale a 0 <= U <= (m-n)
    U = U + n                        # Equivale a n <= U <= m
    return U


def permutar(a):
    """
    Permuta el arreglo a
    args: 
        a: Arreglo a permutar
    return:
        a_p: Arreglo permutado
    """
    N = len(a)
    a_p = a.copy()
    for i in range(N):
        U = uniforme_d(i, N-1)             # Numero aleatorio entre [i, N-1]
        a_p[U], a_p[i] = a_p[i], a_p[U]    # Permutamos un elemento del arreglo
    return a_p


### Funciones generales (Test para datos discretos)


In [44]:
def calcular_T(N, p):
    """
    Calcula el estadistico T dado en el test chi-cuadrado de 
    Pearson.
    ¡Ambas entradas deben estar ordenadas de menor a mayor respecto
    al valor de variable aleatoria al que hacen referencia!
    Args:
        N:    Frecuencia observada de la muestra
        p:    Probabilidad de los valores posibles de la variable aleatoria
    """
    n = sum(N)    # Tamaño de la muestra
    T = sum((N - n*p)**2 / (n*p))
    return T


def p_valor_pearson(t, grados_libertad):
    """
    Calcula el p-valor usando los resultados teoricos que dictan:
    Ph0(T >= t) = P(X_k-1 >= t)
    Args:
        t              :    Resultado del estadistico en la muestra
        grados_libertad:    Grados de libertad de la variable chi-cuadrada
    """
    acumulada_chi = chi2.cdf(t, grados_libertad)
    p_valor = 1 - acumulada_chi
    return p_valor


def generar_frecuencia_observada(n, p_masa, k):
    """
    Genera la frecuencia observada usando una variable aleatoria binomial
    Args:
        n     :   Tamaño de la muestra
        p_masa:   Probabilidad masa de los valores posibles de la distribucion 
                  de la hipotesis nula
        k     :   Cantidad total de valores posibles de la variable con distribucion
                  dada en la hipotesis nula
    """
    N = np.zeros(k)
    N[0] = generar_binomial(n, p_masa[0])
    for i in range(1, k-1):
        N[i] = generar_binomial(
            n - sum(N[:i]), p_masa[i] / (1 - sum(p_masa[:i])))
    N[k-1] = n - sum(N[:k-1])
    return np.array(N)


def estimar_p_valor_pearson(t, Nsim, p, k, n):
    """
    Estima el p_valor por medio de una simulacion P(T >= t)
    Args:
        t   :   Valor del estadistico en la muestra original
        Nsim:   Cantidad de simulaciones a realizar
        p   :   Probabilidad masa de los valores posibles de la distribucion 
                de la hipotesis nula
        k   :   Cantidad total de valores posibles de la variable con distribucion
                dada en la hipotesis nula
        n   :   Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        # Frecuencia observada en la simulacion
        N_sim = generar_frecuencia_observada(n, p, k)
        # Estadistico de la simulacion
        t_sim = calcular_T(N_sim, p)
        if t_sim >= t:
            p_valor += 1
    return p_valor / Nsim


### Funciones generales (Test para datos continuos)


In [45]:
def calcular_D(muestra, acumulada_h):
    """
    Calcula el estadistico dado en el test de Kolmogorov-Smirnov
    Args:
        muestra    :    Muestra original de datos
        acumulada_h:    Acumulada de la distribucion de la hipotesis nula
    """
    n = len(muestra)
    m_copy = list.copy(muestra)
    m_copy.sort()
    m_copy = np.array(m_copy)

    # Vectores_auxiliares
    F = np.array(list(map(acumulada_h, m_copy)))
    j = np.array(list(range(1, n+1))) / n
    j_1 = j - 1/n

    m1 = max(j - F)
    m2 = max(F - j_1)
    D = max(m1, m2)
    return D


def acumulada_uniforme(x):
    """
    Acumulada de la distribucion uniforme (U(0, 1))
    """
    if x < 0:
        return 0
    elif x < 1:
        return x
    else:
        return 1


def estimar_p_valor_k(d, Nsim, n):
    """
    Estima el p-valor por medio de simulacion P(D >= d)
    args:
        d   :    Valor del estadistico en la muestra original
        Nsim:    Cantidad de simulaciones a realizar
        n   :    Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        muestra_sim = generar_muestra(random, n)    # Muestra de uniformes
        muestra_sim.sort()
        d_sim = calcular_D(muestra_sim, acumulada_uniforme)
        if d_sim >= d:
            p_valor += 1
    return p_valor / Nsim


### Ejercicio 1


In [46]:
# Datos de la muestra
# Mapeamos a cada flor con un valor discreto asi podemos modelarla con una variable
# aleatoria
# Flor Blanca ----> 0
# Flor Rosa   ----> 1
# Flor Roja   ----> 2
N_1 = np.array([141, 291, 132])    # Frecuencia observada.
# Masa de nuestra distribucion de la hipotesis nula.
p_1 = np.array([1/4, 1/2, 1/4])
n_1 = 564                          # Tamaño de la muestra
t_1 = calcular_T(N_1, p_1)         # Valor del estadisctico en la muestra
# Cantidad de valores posibles de la muestra (dado por H0)
k_1 = 3
print(N_1)
# Aproximo el p-valor
# Usando la prueba de pearson
p_valor_1 = p_valor_pearson(t_1, grados_libertad=k_1 - 1)

# Usando una simulacion
Nsim = 10**3
p_valor_est_1 = estimar_p_valor_pearson(t_1, Nsim, p_1, k_1, n_1)

data_1 = [[p_valor_1, p_valor_est_1]]
headers_1 = ["P-valor Pearson", "P-valor simulacion"]
print(tabulate(data_1, headers=headers_1, tablefmt="pretty"))


[141 291 132]
+--------------------+--------------------+
|  P-valor Pearson   | P-valor simulacion |
+--------------------+--------------------+
| 0.6499557054800363 |       0.648        |
+--------------------+--------------------+


### Ejercicio 2


In [47]:
# Datos de la muestra
N_2 = np.array([158, 172, 164,     # Frecuencia observada.
                181, 160, 165])
# Masa de nuestra distribucion de la hipotesis nula.
p_2 = np.array([1/6] * 6)
n_2 = 1000                         # Tamaño de la muestra
t_2 = calcular_T(N_2, p_2)         # Valor del estadisctico en la muestra
k_2 = 6                            # Cantidad de valores posibles en la muestra

# Aproximo el p-valor
# Usando la prueba de pearson
p_valor_2 = p_valor_pearson(t_2, grados_libertad=k_2 - 1)

# Usando una simulacion
Nsim = 10**3
p_valor_est_2 = estimar_p_valor_pearson(t_2, Nsim, p_2, k_2, n_2)

data_2 = [[p_valor_2, p_valor_est_2]]
headers_2 = ["P-valor Pearson", "P-valor simulacion"]
print(tabulate(data_2, headers=headers_2, tablefmt="pretty"))


+--------------------+--------------------+
|  P-valor Pearson   | P-valor simulacion |
+--------------------+--------------------+
| 0.8237195392577814 |       0.803        |
+--------------------+--------------------+


### Ejercicio 3


In [48]:
# Datos de la muestra
muestra_3 = [0.12, 0.18, 0.06, 0.33, 0.72, 0.83, 0.36, 0.27, 0.77, 0.74]

d_3 = calcular_D(muestra_3, acumulada_uniforme)    # Valor del estadistico
n_3 = len(muestra_3)                               # Tamaño de la muestra

# Estimamos el p-valor usando simulaciones
Nsim = 10**4
p_valor_3 = estimar_p_valor_k(d_3, Nsim, n_3)

data_3 = [[p_valor_3, d_3]]
headers_3 = ["p-valor estimado", "Estadistico"]

print(tabulate(data_3, headers=headers_3, tablefmt="pretty"))


+------------------+-------------+
| p-valor estimado | Estadistico |
+------------------+-------------+
|      0.5418      |    0.24     |
+------------------+-------------+


### Ejercicio 4


In [49]:
def acumulada_4(x):
    """
    Acumulada de la distribucion dada en la hipotesis nula (e(1/50))
    """
    if x < 0:
        return 0
    else:
        Lambda = 1/50
        return 1 - exp(-Lambda * x)


In [50]:
# Datos de la muestra
muestra_4 = [86, 133, 75, 22, 11, 144, 78, 122, 8, 146, 33, 41, 99]
d_4 = calcular_D(muestra_4, acumulada_4)    # Valor del estadistico
n_4 = len(muestra_4)                        # Tamaño de la muestra

# Estimamos el p-valor usando simulaciones
Nsim = 10**4
p_valor_4 = estimar_p_valor_k(d_4, Nsim, n_4)

data_4 = [[p_valor_4, d_4]]
headers_4 = ["p-valor estimado", "Estadistico"]

print(tabulate(data_4, headers=headers_4, tablefmt="pretty"))


+------------------+--------------------+
| p-valor estimado |    Estadistico     |
+------------------+--------------------+
|      0.0268      | 0.3922544552361856 |
+------------------+--------------------+


### Ejercicio 5


In [51]:
def masa_5(x, p):
    """
    Funcion de probabilidad de masa de la distribucion dada en la 
    hipotesis nula en el ejercicio 5 
    """
    N = 8   # Parametro de la binomial
    if x < 0:
        return 0
    else:
        return comb(N, x) * p**x * (1-p)**(N-x)


In [52]:
# Datos de la muestra
muestra_5 = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
n_5 = len(muestra_5)                # Tamaño de la muestra
N_5 = [0, 1, 2, 4, 1, 1, 2, 5, 2]   # Frecuencia observada
k_5 = 9                             # Valores posibles en la muestra

# Como hipotesis tenemos que los provienen de una variable binomial Bin(n=8, p)
# con el parametro p desconocido, por lo cual debemos estimarlo
p_5_est = sum(muestra_5) / n_5      # Estimacion de la esperanza bin(8, p) = 8*p
p_5_est = p_5_est / 8               # Estimacion de p

# Vector que contiene las probabilidades teoricas para cada valor posible de la
# variable aleatoria 
p_5 = np.array(list(map(lambda x: masa_5(x, p_5_est), list(range(0, 9)))))      

# Ahora podemos calcular T
t_5 = calcular_T(N_5, p_5)

# Aproximamos el p-valor
# Usando la prueba de Pearson
p_valor_5 = p_valor_pearson(t_5, grados_libertad = k_5 - 2)   


# Usando simulacion
Nsim = 10**4
p_valor_5_sim = 0
for _ in range(Nsim):
    # Muestra generada a partir de la distribucion con el parametro estimado
    # en la muestra original
    muestra_sim = generar_muestra(lambda: generar_binomial(8, p_5_est), n_5)
    
    # Re-estimamos el parametro p pero ahora con esta muestra
    p_sim_est = sum(muestra_sim) / len(muestra_sim) / 8
    
    # Calculamos la frecuencia observada en la muestra
    N_sim = np.zeros(9)
    for dato in muestra_sim:
        N_sim[dato] += 1
        
    # Calculamos la probabilidad de masa teorica de cada valor posible con el parametro
    # estimado con la muestra de la simulacion
    p_sim = np.array(list(map(lambda x: masa_5(x, p_sim_est), list(range(0, 9)))))
    # Calculamos el valor del estadistico de la simulacion
    t_sim = calcular_T(N_sim, p_sim)

    if t_sim > t_5:
        p_valor_5_sim += 1
        
p_valor_5_sim = p_valor_5_sim / Nsim

datos_5   = [[t_5, p_valor_5, p_valor_5_sim]]
headers_5 = ["Estadistico T", "p-valor por Pearson", "p-valor simulado"]

print(tabulate(datos_5, headers=headers_5, tablefmt="pretty"))

+--------------------+------------------------+------------------+
|   Estadistico T    |  p-valor por Pearson   | p-valor simulado |
+--------------------+------------------------+------------------+
| 31.499330934155324 | 5.0279943204278865e-05 |      0.0094      |
+--------------------+------------------------+------------------+


### Ejercicio 6


In [53]:
N_6 = [188, 138, 87, 65, 48,        # Frecuencia observada
       32,  30, 34, 13,  2]
k_6 = 10                            # Cantidad posible de valores de la distribucion H0
p_6 = np.array([0.31, 0.22, 0.12,   # Probabilidad de masa para cada valor de la hipotesis
                0.10, 0.08, 0.06,
                0.04, 0.04, 0.02,
                0.01])
t_6 = calcular_T(N_6, p_6)          # Estadistico de la muestra original
n_6 = sum(N_6)                      # Tamaño de la muestra original

# Estimamos el p-valor
# Usando la prueba de Pearson
p_valor_6 = p_valor_pearson(t_6, grados_libertad=k_6-1)

# Usando simulacion
Nsim = 10**4
p_valor_6_sim = estimar_p_valor_pearson(t_6, Nsim, p_6, k_6, n_6)

data_6 = [[p_valor_6, p_valor_6_sim, t_6]]
headers_6 = ["p-valor Pearson", "p-valor Simulacion", "Estadistico"]

print(tabulate(data_6, headers=headers_6, tablefmt="pretty"))


+---------------------+--------------------+-------------------+
|   p-valor Pearson   | p-valor Simulacion |    Estadistico    |
+---------------------+--------------------+-------------------+
| 0.36605389988682613 |       0.3533       | 9.810370888711903 |
+---------------------+--------------------+-------------------+


### Ejercicio 7


In [54]:
def acumulada_7(x):
    """
    Acumulada de la distribucion dada en la hipotesis nula (e(1))
    """
    if x < 0:
        return 0
    else:
        Lambda = 1
        return 1 - exp(-Lambda * x)


In [68]:
# Generamos una muestra de 10 valores de una exponencial(1)
muestra_7 = generar_muestra(lambda: generar_exp(1), 10)
n_7 = 10                                   # Tamaño de la muestra
d_7 = calcular_D(muestra_7, acumulada_7)   # Estadistico de Kolmogorov-Smirnov
print(d_7)

# Estimamos el p_valor
Nsim = 10**4
p_valor_7 = estimar_p_valor_k(d_7, Nsim, n_7)

data_7 = [[d_7, p_valor_7]]
headers_7 = ["Valor del estadistico", "P-valor estimado"]

print(tabulate(data_7, headers=headers_7, tablefmt="pretty"))


[0.2727424247683454, 0.5192651264663054, 2.107705961978529, 0.03792379875403855, 0.6921926213044001, 2.310402081391086, 0.0579246808262929, 1.6272924469195063, 1.4341344211560354, 0.2136124516397159]
0.1616784391368581
+-----------------------+------------------+
| Valor del estadistico | P-valor estimado |
+-----------------------+------------------+
|  0.1616784391368581   |      0.9214      |
+-----------------------+------------------+


### Ejercicio 8


In [56]:
def rt(df):
    """
    Genera variables aleatorias de una distribucion t-student
    Args:
        df: Grados de libertad de la distribucion
    """
    x = gauss(0.0, 1.0)
    y = 2.0 * gammavariate(0.5*df, 2.0)
    return x / (sqrt(y/df))

# La acumulada de la distribucion de nuestra hipotesis


def acumulada_z(x):
    """
    Distribucion acumulada de la variable aleatoria normal estandar N(0, 1)
    """
    return erf(x/sqrt(2.))/2.+0.5


In [57]:
# Generamos la muestra de 10, 20, 100 y 1000 elementos de la t_studen de 11 grados
# de libertad
muestra_8_10 = generar_muestra(lambda: rt(df=11), 10)
muestra_8_20 = generar_muestra(lambda: rt(df=11), 20)
muestra_8_100 = generar_muestra(lambda: rt(df=11), 100)
muestra_8_1000 = generar_muestra(lambda: rt(df=11), 1000)

# Calculo los respectivos estadisticos D
d_8_10 = calcular_D(muestra_8_10, acumulada_z)
d_8_20 = calcular_D(muestra_8_20, acumulada_z)
d_8_100 = calcular_D(muestra_8_100, acumulada_z)
d_8_1000 = calcular_D(muestra_8_1000, acumulada_z)

# Estimo los respectivos p-valores
Nsim = 10**3
p_valor_8_10 = estimar_p_valor_k(d_8_10, Nsim, 10)
p_valor_8_20 = estimar_p_valor_k(d_8_20, Nsim, 20)
p_valor_8_100 = estimar_p_valor_k(d_8_100, Nsim, 100)
p_valor_8_1000 = estimar_p_valor_k(d_8_1000, Nsim, 1000)

# Tabla de datos
data_8 = [[10, d_8_10, p_valor_8_10],
          [20, d_8_20, p_valor_8_20],
          [100, d_8_100, p_valor_8_100],
          [1000, d_8_1000, p_valor_8_1000]]

headers_8 = ["Tamaño de la muestra", "Estadistico", "p-valor estimado"]

print(tabulate(data_8, headers=headers_8, tablefmt="pretty"))


+----------------------+---------------------+------------------+
| Tamaño de la muestra |     Estadistico     | p-valor estimado |
+----------------------+---------------------+------------------+
|          10          | 0.19123882714498086 |      0.798       |
|          20          | 0.23246969293848257 |      0.194       |
|         100          | 0.12189128787792924 |      0.081       |
|         1000         | 0.09625357920825578 |       0.0        |
+----------------------+---------------------+------------------+


### Ejercicio 9


In [58]:
def acumulada_9(x, Lambda):
    """
    Acumulada de la distribucion dada en la hipotesis nula en el ejercicio 9 
    """
    if x < 0:
        return 0
    else:
        return 1 - exp(-Lambda * x)


In [59]:
muestra_9 = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3,
             10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]

n_9 = len(muestra_9)   # Tamaño de la muestra

# Como hipotesis nula tenemos que estos datos provienen de una variable
# con distribucion exponencial de parametro desconocido.

# procedemos a estimar ese parametro
Lambda_9_est = sum(muestra_9) / n_9   # Estimamos E[e(λ)] = 1/λ
Lambda_9_est = 1 / Lambda_9_est       # Invertimos y obtenemos λ

# Calculamos el valor del estimador en la muestra original
d_9 = calcular_D(muestra_9, lambda x: acumulada_9(x, Lambda_9_est))

# Estimamos el p-valor por medio de simulaciones sobre uniformes (no es fiable)
Nsim = 10**4
p_valor_9 = estimar_p_valor_k(d_9, Nsim, n_9)

# Estimamos el p-valor con una simulacion mas certera (calculando estadisticos
# de muestras con distribucion H0)
p_valor_9_sim = 0

for _ in range(Nsim):
    # Generamos la muestra a partir de la distribucion "original"
    muestra_sim = generar_muestra(lambda: generar_exp(Lambda_9_est), n_9)
    # Reestimamos el parametro desconocido
    Lambda_sim = n_9 / sum(muestra_sim)
    # Calculamos el estadistico de la simulacion usando el parametro estimado
    d_sim = calcular_D(muestra_sim, lambda x: acumulada_9(x, Lambda_sim))
    if d_sim > d_9:
        p_valor_9_sim += 1

p_valor_9_sim = p_valor_9_sim / Nsim


data_9 = [[Lambda_9_est, p_valor_9, p_valor_9_sim, d_9]]
headers_9 = ["Parametro λ estimado",
             "p-valor (uniformes)", "p-valor (no uniformes)", "Estadistico"]


print(tabulate(data_9, headers=headers_9, tablefmt="pretty"))


+----------------------+---------------------+------------------------+---------------------+
| Parametro λ estimado | p-valor (uniformes) | p-valor (no uniformes) |     Estadistico     |
+----------------------+---------------------+------------------------+---------------------+
| 0.09322560596643878  |       0.1899        |         0.0488         | 0.26949936321059237 |
+----------------------+---------------------+------------------------+---------------------+


### Ejercicio 10


In [60]:
def acumulada_10(x, mu, sigma):
    """
    Distribucion acumulada de la variable aleatoria dada como 
    hipotesis nula en el ejercicio 10
    """
    return erf((x-mu)/(sigma*sqrt(2.)))/2.+0.5


def normal_ross(mu=0, sigma=1):
    """
    Genera una variable normal estandar (por defecto) usando el algoritmo descrito 
    en el libro de Sheldon Ross, el cual esta basado en el metodo de aceptacion y 
    rechazo
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    """
    while True:
        Y1 = -log(random())  # Exponencial de parametro 1
        Y2 = -log(random())  # Exponencial de parametro 1
        if Y2 >= (Y1 - 1) ** 2 / 2:
            # En este punto Y1 es una variable aleatoria con distribucion |Z|
            # La siguiente comparacion es el metodo para generar Z con composicion
            # a partir del generador de |Z|
            if random() < 0.5:
                return Y1 * sigma + mu
            else:
                return -Y1 * sigma + mu


In [61]:
muestra_10 = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0,
              103.8, 99.6,  96.6, 119.3, 104.8, 101.7]

n_10 = len(muestra_10)    # Tamaño de la muestra

# Por la hipotesis nula suponemos que los datos provienen de una variable
# normal la cual desconocemos sus parametros, procedemos a estimarlos

# Estimacion de la media de la normal
mu_10_est = sum(muestra_10) / n_10
sigma_10_est = sum(
    list(map(lambda x: (x - mu_10_est)**2, muestra_10))) / (n_10 - 1)
# Estimacion de la desviacion
sigma_10_est = sqrt(sigma_10_est)

# Calculamos el valor de estimador D
d_10 = calcular_D(muestra_10, lambda x: acumulada_10(
    x, mu_10_est, sigma_10_est))

# Estimamos el p-valor
Nsim = 10**4
p_valor_10 = estimar_p_valor_k(d_10, Nsim, n_10)

# Estimamos el p-valor con una simulacion mas certera (calculando estadisticos
# de muestras con distribucion H0)
p_valor_10_sim = 0

for _ in range(Nsim):
    # Generamos la muestra a partir de la distribucion "original"
    muestra_sim = generar_muestra(
        lambda: normal_ross(mu_10_est, sigma_10_est), n_10)

    # Reestimamos los parametros desconocidos
    mu_sim = sum(muestra_sim) / n_10
    sigma_sim = sqrt(
        sum(list(map(lambda x: (x - mu_sim)**2, muestra_sim))) / n_10)

    # Calculamos el estadistico de la simulacion usando los parametros estimados
    d_sim = calcular_D(
        muestra_sim, lambda x: acumulada_10(x, mu_sim, sigma_sim))

    if d_sim > d_10:
        p_valor_10_sim += 1

p_valor_10_sim = p_valor_10_sim / Nsim


data_10 = [[mu_10_est, sigma_10_est, p_valor_10, p_valor_10_sim]]
headers_10 = ["μ estiamdo", "σ estimado",
              "p-valor (uniformes)", "p-valor (no uniformes)"]


print(tabulate(data_10, headers=headers_10, tablefmt="pretty"))


+--------------------+-------------------+---------------------+------------------------+
|     μ estiamdo     |    σ estimado     | p-valor (uniformes) | p-valor (no uniformes) |
+--------------------+-------------------+---------------------+------------------------+
| 104.13333333333334 | 9.397420664816233 |       0.6737        |         0.2385         |
+--------------------+-------------------+---------------------+------------------------+


### Funciones generales (problemas de dos muestras)


In [62]:
def calcular_R(datos_1, datos_2):
    """
    Calcula el valor del estadistico dado en el test de suma de rangos
    Args:
        datos_1:   Primera muestra
        datos_2:   Segunda muestra
    """
    n = len(datos_1)
    m = len(datos_2)
    R = 0

    datos = np.array(datos_1 + datos_2)   # Lista auxiliar
    orden = np.argsort(datos)             # Indices ordenados

    for i in range(n + m):
        if orden[i] < n:
            R += i + 1
    return R


def rangos(n, m, r):
    """
    Formula recursiva para calcular Pn,m(r) = Ph0(R <= r)
    """
    if n == 1 and m == 0:
        if r < 1:
            p = 0
        else:
            p = 1
    elif n == 0 and m == 1:
        if r < 0:
            p = 0
        else:
            p = 1
    else:
        if n == 0:
            p = rangos(0, m-1, r)
        elif m == 0:
            p = rangos(n-1, 0, r-n)
        else:  # n>0, m>0
            p = (n*rangos(n-1, m, r-n-m)+m*rangos(n, m-1, r))/(n+m)
    return p


def p_valor_recursivo(n, m, r):
    """
    Calcula el p_valor usando la formula recursiva Pn,m(r) = Ph0(R <= r)
    Args:
        n:    Cantidad de elementos en la muestra 1
        m:    Cantidad de elementos en la muestra 2
        r:    Rango de la muestra original
    """
    p_valor = 2 * min(rangos(n, m, r), 1 - rangos(n, m, r-1))
    return p_valor


def p_valor_normal(n, m, r):
    """
    Calcula el p_valor usando la distribucion normal estandar
    Args:
        n:    Cantidad de elementos en la muestra 1
        m:    Cantidad de elementos en la muestra 2
        r:    Rango de la muestra original
    """
    def cdf_z(x): return erf(x/sqrt(2))/2 + \
        0.5           # Acumulada de la normal estandar
    r_star = (r - n*(n+m+1)/2) / sqrt(n*m*(n+m+1)/12)   # r* de la teoria
    p_valor = 2 * min(cdf_z(r_star), 1 - cdf_z(r_star))
    return p_valor


def estimar_p_valor_r(r, Nsim, n, m):
    """
    Estima el p-valor por medio de simulaciones
    Args:
        r   :   Estadistico R de la muestra original 
        Nsim:   Cantidad de simulaciones a realizar
        n   :   Tamaño de la primera muestra
        m   :   Tamaño de la segunda muestra
    """
    mayor_a_r = 0
    menor_a_r = 0
    aux = list(range(1, n+m+1))   # Arreglo para generar estadisticos
    for _ in range(Nsim):
        aux = permutar(aux)
        r_sim = sum(aux[:n])
        if r_sim >= r:
            mayor_a_r += 1
        if r_sim <= r:
            menor_a_r += 1

    p_valor = 2 * min(mayor_a_r, menor_a_r) / Nsim
    return p_valor


### Ejercicio 11


In [63]:
muestra_11_1 = [65.2, 67.1, 69.4, 78.4, 74.0, 80.3]   # Primera muestra
muestra_11_2 = [59.4, 72.1, 68.0, 66.2, 58.5]         # Segunda muestra
r_11 = calcular_R(muestra_11_1, muestra_11_2)         # Estadistico R

n_11 = len(muestra_11_1)    # Tamaño de la primera muestra
m_11 = len(muestra_11_2)    # Tamaño de la segunda muestra

# Calcular el p−valor exacto de este conjunto de datos
p_valor_exacto_11 = p_valor_recursivo(n_11, m_11, r_11)

# Calcular el p−valor aproximado en base a una aproximación normal
p_valor_normal_11 = p_valor_normal(n_11, m_11, r_11)

# Calcular el p−valor aproximado en base a una simulación
Nsim = 10**4
p_valor_sim_11 = estimar_p_valor_r(r_11, Nsim, n_11, m_11)


data_11 = [[p_valor_exacto_11, p_valor_normal_11, p_valor_sim_11]]
headers_11 = ["p-valor exacto", "p-valor (normal)", "p-valor (simulacion)"]
print(tabulate(data_11, headers=headers_11, tablefmt="pretty"))


+--------------------+--------------------+----------------------+
|   p-valor exacto   |  p-valor (normal)  | p-valor (simulacion) |
+--------------------+--------------------+----------------------+
| 0.1255411255411254 | 0.1003482464622909 |        0.1298        |
+--------------------+--------------------+----------------------+


### Ejercicio 12


In [64]:
# A pesar de que no todos los valores sean distintos tenemos la suerte
# de que entre ambas muestras no hay valores en comun, entonces podemos
# usar los algoritmos de siempre sin preocupaciones

muestra_12_1 = [19, 31, 39, 45, 47, 66, 75]     # Primera muestra
muestra_12_2 = [28, 36, 44, 49, 52, 72, 72]     # Segunda muestra
r_12 = calcular_R(muestra_12_1, muestra_12_2)   # Estadistico de la muestra
n_12 = len(muestra_12_1)   # Tamaño de la primera muestra
m_12 = len(muestra_12_2)   # Tamaño de la segunda muestra


# Calcular el p−valor exacto de este conjunto de datos
p_valor_exacto_12 = p_valor_recursivo(n_12, m_12, r_12)

# Calcular el p−valor aproximado en base a una aproximación normal
p_valor_normal_12 = p_valor_normal(n_12, m_12, r_12)

# Calcular el p−valor aproximado en base a una simulación
Nsim = 10**4
p_valor_sim_12 = estimar_p_valor_r(r_12, Nsim, n_12, m_12)


data_12 = [[p_valor_exacto_12, p_valor_normal_12, p_valor_sim_12]]
headers_12 = ["p-valor exacto", "p-valor (normal)", "p-valor (simulacion)"]
print(tabulate(data_12, headers=headers_12, tablefmt="pretty"))


+--------------------+--------------------+----------------------+
|   p-valor exacto   |  p-valor (normal)  | p-valor (simulacion) |
+--------------------+--------------------+----------------------+
| 0.7103729603729604 | 0.6547208460185772 |        0.6988        |
+--------------------+--------------------+----------------------+


### Ejercicio 13


In [65]:
muestra_13_1 = [39, 40, 38.9, 35, 32, 33, 22.8, 36]    # Primera muestra
muestra_13_2 = [36.5, 33.1, 35.2, 30, 29, 26, 35.1]    # Segunda muestra
# Estadistico de la muestra
r_13 = calcular_R(muestra_13_1, muestra_13_2)
n_13 = len(muestra_13_1)    # Tamaño de la primera muestra
m_13 = len(muestra_13_2)    # Tamaño de la segunda muestra


# Calculamos el p-valor exacto (usando recursion)
p_valor_exacto_13 = p_valor_recursivo(n_13, m_13, r_13)

# Calcular el p−valor aproximado en base a una aproximación normal
p_valor_normal_13 = p_valor_normal(n_13, m_13, r_13)

# Calcular el p−valor aproximado en base a una simulación
Nsim = 10**4
p_valor_sim_13 = estimar_p_valor_r(r_13, Nsim, n_13, m_13)

data_13 = [["p-valor (exacto)", p_valor_exacto_13, p_valor_exacto_13 < 0.05],
           ["p-valor (simul.)", p_valor_sim_13, p_valor_sim_13 < 0.05],
           ["p-Valor (normal)", p_valor_normal_13, p_valor_normal_13 < 0.05]]

headers_13 = ["Caso", "Estimacion", "¿Rechazar H0 al 5% de rechazo?"]

print(tabulate(data_13, headers=headers_13, tablefmt="pretty"))


+------------------+---------------------+--------------------------------+
|       Caso       |     Estimacion      | ¿Rechazar H0 al 5% de rechazo? |
+------------------+---------------------+--------------------------------+
| p-valor (exacto) | 0.33566433566433584 |             False              |
| p-valor (simul.) |       0.3342        |             False              |
| p-Valor (normal) |  0.297621467891116  |             False              |
+------------------+---------------------+--------------------------------+


### Ejercicio 14


In [66]:
muestra_14_1 = [141, 132, 154, 142, 143, 150, 134, 140]   # Primera muestra
muestra_14_2 = [133, 138, 136, 125, 135, 130, 127, 131,   # Segunda muestra
                116, 128]
# Estadistico de la muestra
r_14 = calcular_R(muestra_14_1, muestra_14_2)
n_14 = len(muestra_14_1)    # Tamaño de la primera muestra
m_14 = len(muestra_14_2)    # Tamaño de la segunda muestra

# Calculo del p-valor exacto (recursion)
p_valor_exacto_14 = p_valor_recursivo(n_14, m_14, r_14)

# Calculo del p-valor usando una Normal
p_valor_normal_14 = p_valor_normal(n_14, m_14, r_14)

# Calcular el p−valor aproximado en base a una simulación
Nsim = 10**4
p_valor_sim_14 = estimar_p_valor_r(r_14, Nsim, n_14, m_14)


data_14 = [["p-valor (exacto)", p_valor_exacto_14, p_valor_exacto_14 < 0.05],
           ["p-valor (simul.)", p_valor_sim_14, p_valor_sim_14 < 0.05],
           ["p-Valor (normal)", p_valor_normal_14, p_valor_normal_14 < 0.05]]

headers_14 = ["Caso", "Estimacion", "¿Rechazar H0 al 5% de rechazo?"]

print(tabulate(data_14, headers=headers_14, tablefmt="pretty"))


+------------------+----------------------+--------------------------------+
|       Caso       |      Estimacion      | ¿Rechazar H0 al 5% de rechazo? |
+------------------+----------------------+--------------------------------+
| p-valor (exacto) | 0.002056766762648987 |              True              |
| p-valor (simul.) |        0.0016        |              True              |
| p-Valor (normal) | 0.003366491079945577 |              True              |
+------------------+----------------------+--------------------------------+
